In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
#r ".\binaries\ApplicationWithIDT.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using ApplicationWithIDT;
using XESF;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


# Run Simulation

open a database

In [2]:
string dbPath = @"WedgeFlow_db";
var database = OpenOrCreateDatabase(dbPath);
database

Opening existing database 'WedgeFlow_db'.


{ Session Count = 5; Grid Count = 5; Path = WedgeFlow_db }

run WedgeFlow simulation and safe to database

In [3]:
BoSSS.Solution.Application.InitMPI(num_threads:1);
BoSSS.Solution.Application.DeleteOldPlotFiles();
var p = new XESFMain();

var C = XESFHardCodedControl.XDGWedgeFlow_TwoLs_Base(
                    dbPath:dbPath,
                    optiLSDegree: 1,
                    lsDegree: 1,
                    shocksetup: ApplicationWithIDT.GetLevelSet.FromFunction,
                    optiLevelSetType: OptiLevelSetType.SplineLevelSet,
                    initialValue: ApplicationWithIDT.GetInitialValue.FromFunctionPerSpecies,
                    MaxIterations: 200,
                    dgDegree: 0,
                    numOfCellsX: 15,
                    numOfCellsY: 10,
                    initialAngle_shockLS: 32,
                    PlotInterval: -1,
                    interfaceFluxLS2: XESF.Fluxes.ConvectiveInterfaceFluxes.GodunovInterface,
                    bulkFlux: XESF.Fluxes.ConvectiveBulkFluxes.OptimizedHLLC,
                    FluxVersion: XESF.Fluxes.FluxVersion.Optimized,
                    agg: 0.4,
                    globalization: ApplicationWithIDT.GlobalizationStrategy.LineSearch
                    );   
p.Init(C);
p.RunSolverMode(); 
//p.InitializeEverything();

rm XESFWedgeFlow_TwoLs_0.plt;
Grid Edge Tags changed.
Session ID: b62549af-c208-46f2-84be-a8b3c1e5bcfc, DB path: 'WedgeFlow_db'
Session directory 'WedgeFlow_db\sessions\b62549af-c208-46f2-84be-a8b3c1e5bcfc'.
Grid repartitioning method: METIS
Grid repartitioning options: 
Number of cell Weights: 0


*****************************************************************************************
*  X   X   DDD   GGG   -   III  SSS  TTTTT  *
*   X X    D  D G      -    I  S      T     *
*    X     D  D G  GG  -    I   SSS   T     *      XDG Implicit Shock Tracking Solver    
*   X X    D  D G   G  -    I      S  T     *
*  X   X   DDD   GGG   -   III  SSS   T     *
*****************************************************************************************
*  
*  solver:                             XESF.XESFMain
*  project:                            XESFWedgeFlow_TwoLs
*  session:                            WedgeFlow_TwoLs_p0_xCells15_yCells10_GodunovInterface_iterations200_angle32_lSDegree1_GlobL

# Postprocessing

we need to load something from the ApplicationWithIDT assembly 

In [18]:
string dbPath = @"WedgeFlow_db";
var database = OpenOrCreateDatabase(dbPath);
MatrixAssembler aaa = new MatrixAssembler(3);
var si = database.Sessions.Pick(2);
si

XESFWedgeFlow_TwoLs	WedgeFlow_TwoLs_p0_xCells15_yCells10_GodunovInterface_iterations200_angle32_lSDegree1_GlobLineSearch_agg0.4_*	4/17/2024 9:55:43 AM	ac1d2651...

## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(\gamma -1)(\rho E - \rho e_{kin}),~\rho e_{kin}=\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$

For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the WedgeFlow ($p_\text{in}=1,\rho_\text{in}=1$) presented, the v-velocity is zero and we have
$$ h_{in} = \rho E_{in}+1$$
We prescribe the inflow conditions by the Mach number $M_{in}=2$. From those the Energy is computed as 
$$ \rho E_{in} = (\rho e_{inner})_\text{in}+(\rho e_{kin})_\text{in}=\frac{p_\text{in}}{\gamma-1}+\frac{\rho_\text{in}}{2}u_\text{in}^2= \frac{1}{\gamma-1}+\frac{1}{2}u_{in}^2$$ 
where $u_{in} = M_{in}\sqrt{\gamma}$. In total we obtain:
$$ h_\text{in}=\rho E_{in}+1=\frac{\gamma}{\gamma-1}+\frac{1}{2}u_{in}^2$$


In [19]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 2;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

6.300000000000001

In [20]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

In [21]:
p.GetResEnthalpyPlot(si,EEN).ToGnuplot().PlotSVG(xRes:1200,yRes:500)

Error: System.ArgumentException: Number of x and y values must be identical within each group
   at BoSSS.Solution.Gnuplot.Plot2Ddata.XYvalues..ctor(String name, IEnumerable`1 abscissas, IEnumerable`1 values) in C:\experimental\public\src\L3-solution\BoSSS.Solution.Gnuplot\Plot2Ddata.cs:line 111
   at BoSSS.Solution.Gnuplot.Plot2Ddata.AddDataGroup(String name, IEnumerable`1 Abscissas, IEnumerable`1 values, PlotFormat Format) in C:\experimental\public\src\L3-solution\BoSSS.Solution.Gnuplot\Plot2Ddata.cs:line 227
   at ApplicationWithIDT.ApplicationWithIDT`1.GetPlot(List`1 y1, String l1, List`1 y2, String l2, List`1 y3, String l3) in C:\experimental\public\src\L4-application\ApplicationWithIDT\ApplicationWithIDT.cs:line 4045
   at ApplicationWithIDT.ApplicationWithIDT`1.GetResEnthalpyPlot(ISessionInfo si, Double EEN) in C:\experimental\public\src\L4-application\ApplicationWithIDT\ApplicationWithIDT.cs:line 4067
   at Submission#22.<<Initialize>>d__0.MoveNext()
--- End of stack trac

In [22]:
p.PlotShadowFields(si);